In [1]:
# Import all required libraries
import pandas as pd

### Exploratory Data Analysis

This part of my work mainly focuses on understanding the underlying data and various relationships between different attributes


In [10]:
df = pd.read_csv('data/train_file.csv')
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,0,nonexistent,yes
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,0,nonexistent,no
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32905,30,management,single,university.degree,no,no,no,cellular,nov,thu,197,5,1,failure,no
32906,34,blue-collar,married,high.school,no,yes,no,telephone,jun,mon,562,3,0,nonexistent,no
32907,33,technician,married,professional.course,no,yes,yes,cellular,aug,mon,61,1,0,nonexistent,no
32908,58,unemployed,married,basic.9y,no,no,no,telephone,jun,mon,266,2,0,nonexistent,no


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32910 entries, 0 to 32909
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          32910 non-null  int64 
 1   job          32910 non-null  object
 2   marital      32910 non-null  object
 3   education    32910 non-null  object
 4   default      32910 non-null  object
 5   housing      32910 non-null  object
 6   loan         32910 non-null  object
 7   contact      32910 non-null  object
 8   month        32910 non-null  object
 9   day_of_week  32910 non-null  object
 10  duration     32910 non-null  int64 
 11  campaign     32910 non-null  int64 
 12  previous     32910 non-null  int64 
 13  poutcome     32910 non-null  object
 14  y            32910 non-null  object
dtypes: int64(4), object(11)
memory usage: 3.8+ MB


##### Problem with Categorical Attributes

<p>A quick glance reveals that in the dataset, all categorical attributes are stored as text data (i.e., objects). This kind of mismatch is overwhelmingly frequent in real-world data. In this case, columns that describe a category to which a targeted person belongs include: </p>

<p>1. <b>job</b>: Category to which a person's job falls into (e.g., 'admin.', 'blue-collar', 'entrepreneur', etc.)</p>
<p>2. <b>marital</b>: Marital status (e.g., 'married', 'single', 'divorced')</p>
<p>3. <b>education</b>: Level of education that a contacted person has achieved so far (e.g., 'secondary', 'tertiary', 'primary', etc.)</p>
<p>4. <b>Financial Status</b>:
    <ul>
        <li><b>default</b>: Column indicating whether a person had credit defaults in the past? (e.g., 'yes', 'no')</li>
        <li><b>housing</b>: Column indicating whether an individual has housing loan/has property? (e.g., 'yes', 'no')</li>
        <li><b>loan</b>: Column indicating whether an individual has any loans? (e.g., 'yes', 'no')</li>
    </ul>
</p>
<p>5. <b>contact</b>: Contacted person's phone type (e.g., 'cellular', 'telephone')</p>
<p>6. <b>Time when a person was contacted</b>:
    <ul>
        <li><b>month</b>: Month when contact took place (e.g., 'jan', 'feb', 'mar', ..., 'dec')</li>
        <li><b>day_of_week</b>: Weekday when contact took place (e.g., 'mon', 'tue', 'wed', 'thu', 'fri')</li>
    </ul>
</p>
<p>7. <b>poutcome</b>: Outcome of the previous marketing campaign? (e.g., 'success', 'failure', 'other', 'unknown')</p>

<p>These categorical attributes might be crucial for understanding customer profiles and predicting subscription likelihood.</p>